In [1]:
#https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE#dependencies_prerequisites
import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K


input_shape = (50, 50, 3)
num_features = 7500#50*50*3
latent_dim = 32

In [ ]:
def encoder_CNN(input_shape = (50, 50, 3), latent_dim = 2):

    inputs = layers.Input(shape=input_shape, name='Input')

    # layer 1
    x = layers.Conv2D(filters=6, kernel_size=5, strides=1,padding='valid',name='C1')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.AvgPool2D(pool_size=2, strides=2,name='S2')(x)

    # layer 2
    x = layers.Conv2D(filters=16, kernel_size=5,strides=1,padding='valid',name='C3')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.AvgPool2D(pool_size=2, strides=2,name='S4')(x)

    # layer 3
    x = layers.Conv2D(filters=120, kernel_size=5,strides=1,padding='valid',name='C5')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Flatten()(x)

    y = layers.Dense(84, activation ='relu',name='F6')(x)
    z_mean = layers.Dense(latent_dim, name='z_mean')(y)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(y)
    z = layers.Lambda(sampling, name='z')([z_mean, z_log_var]) #reparametrization trick
    model = keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')
    
    return model
    

In [ ]:
encoder = encoder_CNN(latent_dim = latent_dim)
encoder.summary()

In [ ]:
def decoder_CNN(input_shape = (50, 50, 3), latent_dim = 2):
    latent_inputs = layers.Input(shape=(latent_dim,), name='z_sampling')
    x = layers.Dense(84, activation ='tanh',name='F6l')(latent_inputs)
    x = layers.Dropout(0.3)(latent_inputs)
    x = layers.Dense(5*5*120, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Reshape(target_shape=(5, 5, 120))(x) #(5, 5, 120)
    x = layers.UpSampling2D((2,2), name='S4l')(x) # (10, 10, 120)
    x = layers.Conv2DTranspose(filters=16, kernel_size=5,strides=1,padding='valid',name='C5l')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2DTranspose(filters=6, kernel_size=5, strides=1,padding='valid',name='C1l')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    outputs = layers.Conv2DTranspose(filters=3, kernel_size=19, strides=1,activation='sigmoid',padding='valid')(x)
   # outputs = layers.Reshape(target_shape=(50, 50, 3), name='output')(x)
    model = keras.Model(latent_inputs, outputs, name='decoder')
    return model

In [ ]:
decoder = decoder_CNN(latent_dim = latent_dim)
decoder.summary()

In [ ]:
def VAE(input_shape, latent_dim, encoder, decoder):
    vae_input = layers.Input(shape = input_shape, name="VAE_input")
    encoder_output = encoder(vae_input)
    decoder_output = decoder(encoder_output[2])
    model = keras.Model(vae_input, decoder_output, name='VAE')
    return model
    

In [ ]:
vae = VAE(input_shape, latent_dim, encoder, decoder)

In [ ]:
vae.summary()

In [ ]:
plot_model(vae,show_shapes=True, show_layer_names=True,expand_nested=True)

In [ ]:
def vae_loss(encoder_mu, encoder_log_variance, beta_coefficient):

    def vae_reconstruction_loss(y_true, y_predict):
        reconstruction_loss_factor = 1000
        reconstruction_loss = K.mean(K.square(y_true - y_predict), axis=[1, 2, 3])
        return reconstruction_loss_factor * reconstruction_loss

    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * K.sum(1.0 + encoder_log_variance - K.square(encoder_mu) - K.exp(encoder_log_variance), axis=[1, 2, 3])
        return kl_loss

    def vae_kl_loss_metric(y_true, y_predict):
        kl_loss = -0.5 * K.backend.sum(1.0 + encoder_log_variance - K.square(encoder_mu) - K.exp(encoder_log_variance), axis=[1, 2, 3])
        return kl_loss

    def vae_loss(y_true, y_predict, beta_coefficient):
        reconstruction_loss = vae_reconstruction_loss(y_true, y_predict)
        kl_loss = vae_kl_loss(y_true, y_predict)

        loss = reconstruction_loss + beta_coefficient * kl_loss
        return loss

    return vae_loss

In [ ]:
beta_coefficient=1

#Information needed to compute the loss function
vae_input = vae.input
vae_output = vae.output
mu = encoder.get_layer('z_mean').output
log_var= encoder.get_layer('z_log_var').output

vae.add_loss(vae_loss( mu, log_var, beta_coefficient))


In [ ]:
vae.compile(optimizer='adam')

In [ ]:
vae.save('models/vae.h5')  
encoder.save('models/encoder.h5')
decoder.save('models/decoder.h5')

In [ ]:
vae.compile(optimizer='adam')